In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical

In [2]:
# =====Read data======
# Dataset link-> https://www.kaggle.com/sulianova/cardiovascular-disease-dataset
rootPath='DS1/'
dfDS = pd.read_csv(rootPath+"cardio.csv", delimiter=';')

In [3]:
#=====Feature Scaling======
columnsToScale = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
scaler = MinMaxScaler(feature_range=(0,1))
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

In [4]:
#=====Split Dataset=======
X = dfDS.iloc[:, 1:len(dfDS.columns)-1].values
Y = dfDS["cardio"].values
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.20)

In [5]:
verbose, epochs, batch_size = 1, 100, 512
activationFunction='relu'

def getModel():
    model = Sequential()
    model.add(Dense(100, input_dim = xTrain.shape[1], activation=activationFunction))
    model.add(Dense(75, activation=activationFunction))
    model.add(Dense(50, activation=activationFunction))
    model.add(Dense(25, activation=activationFunction))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = getModel()

In [6]:
def showResults(test, pred):
    target_names = ['positive', 'negative']
    print(classification_report(test, pred, target_names=target_names))
    accuracy = accuracy_score(test, pred)
    precision=precision_score(test, pred, average='weighted')
    f1Score=f1_score(test, pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(test, pred)
    print(cm) 

In [7]:
#======== Cross Validation ===========
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(xTrain, yTrain)
foldNum=0
for train_index, val_index in skf.split(xTrain, yTrain):
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    
    # one hot encode
    Y_train = to_categorical(Y_train)
    Y_val = to_categorical(Y_val)
    
    history = model.fit(X_train, Y_train, 
                        validation_data = (X_val, Y_val), 
                        epochs=epochs, 
                        batch_size=batch_size)  
    yPredict = model.predict(X_val)

    #Converting one hot encoded test label to label    
    pred = np.argmax(yPredict, axis=1)
    val = np.argmax(Y_val, axis=1)
    
    showResults(val, pred)

Results for fold 1
Train on 44800 samples, validate on 11200 samples
Epoch 1/100
44800/44800 [==============================] - 1s 13us/sample - loss: 0.6612 - accuracy: 0.6002 - val_loss: 0.6409 - val_accuracy: 0.6364
Epoch 2/100
44800/44800 [==============================] - 0s 6us/sample - loss: 0.6391 - accuracy: 0.6332 - val_loss: 0.6315 - val_accuracy: 0.6472
Epoch 3/100
44800/44800 [==============================] - 0s 6us/sample - loss: 0.6329 - accuracy: 0.6403 - val_loss: 0.6276 - val_accuracy: 0.6535
Epoch 4/100
44800/44800 [==============================] - 0s 5us/sample - loss: 0.6326 - accuracy: 0.6411 - val_loss: 0.6301 - val_accuracy: 0.6449
Epoch 5/100
44800/44800 [==============================] - 0s 5us/sample - loss: 0.6304 - accuracy: 0.6446 - val_loss: 0.6295 - val_accuracy: 0.6464
Epoch 6/100
44800/44800 [==============================] - 0s 6us/sample - loss: 0.6311 - accuracy: 0.6401 - val_loss: 0.6312 - val_accuracy: 0.6393
Epoch 7/100
44800/44800 [===========

44800/44800 [==============================] - 1s 33us/sample - loss: 0.5576 - accuracy: 0.7209 - val_loss: 0.5727 - val_accuracy: 0.7057
Epoch 7/100
44800/44800 [==============================] - 1s 29us/sample - loss: 0.5603 - accuracy: 0.7174 - val_loss: 0.5508 - val_accuracy: 0.7287
Epoch 8/100
44800/44800 [==============================] - 1s 31us/sample - loss: 0.5536 - accuracy: 0.7233 - val_loss: 0.5540 - val_accuracy: 0.7244
Epoch 9/100
44800/44800 [==============================] - 1s 33us/sample - loss: 0.5506 - accuracy: 0.7276 - val_loss: 0.5705 - val_accuracy: 0.7080
Epoch 10/100
44800/44800 [==============================] - 1s 25us/sample - loss: 0.5578 - accuracy: 0.7200 - val_loss: 0.5628 - val_accuracy: 0.7176
Epoch 11/100
44800/44800 [==============================] - 1s 30us/sample - loss: 0.5540 - accuracy: 0.7236 - val_loss: 0.5733 - val_accuracy: 0.7018
Epoch 12/100
44800/44800 [==============================] - 1s 33us/sample - loss: 0.5490 - accuracy: 0.7287 -

Epoch 59/100
44800/44800 [==============================] - 1s 22us/sample - loss: 0.5466 - accuracy: 0.7298 - val_loss: 0.5553 - val_accuracy: 0.7238
Epoch 60/100
44800/44800 [==============================] - 1s 24us/sample - loss: 0.5499 - accuracy: 0.7271 - val_loss: 0.5530 - val_accuracy: 0.7269
Epoch 61/100
44800/44800 [==============================] - 1s 23us/sample - loss: 0.5466 - accuracy: 0.7302 - val_loss: 0.5548 - val_accuracy: 0.7239
Epoch 62/100
44800/44800 [==============================] - 1s 26us/sample - loss: 0.5450 - accuracy: 0.7295 - val_loss: 0.5598 - val_accuracy: 0.7197
Epoch 63/100
44800/44800 [==============================] - 1s 23us/sample - loss: 0.5482 - accuracy: 0.7277 - val_loss: 0.5515 - val_accuracy: 0.7274
Epoch 64/100
44800/44800 [==============================] - 1s 24us/sample - loss: 0.5571 - accuracy: 0.7171 - val_loss: 0.5803 - val_accuracy: 0.7004
Epoch 65/100
44800/44800 [==============================] - 1s 25us/sample - loss: 0.5475 - ac

Epoch 63/100
44800/44800 [==============================] - 1s 29us/sample - loss: 0.5421 - accuracy: 0.7299 - val_loss: 0.5329 - val_accuracy: 0.7425
Epoch 64/100
44800/44800 [==============================] - 1s 26us/sample - loss: 0.5432 - accuracy: 0.7300 - val_loss: 0.5487 - val_accuracy: 0.7261
Epoch 65/100
44800/44800 [==============================] - 1s 25us/sample - loss: 0.5440 - accuracy: 0.7298 - val_loss: 0.5486 - val_accuracy: 0.7285
Epoch 66/100
44800/44800 [==============================] - 1s 24us/sample - loss: 0.5449 - accuracy: 0.7296 - val_loss: 0.5497 - val_accuracy: 0.7276
Epoch 67/100
44800/44800 [==============================] - 1s 24us/sample - loss: 0.5438 - accuracy: 0.7300 - val_loss: 0.5419 - val_accuracy: 0.7338
Epoch 68/100
44800/44800 [==============================] - 1s 26us/sample - loss: 0.5466 - accuracy: 0.7283 - val_loss: 0.5482 - val_accuracy: 0.7284
Epoch 69/100
44800/44800 [==============================] - 1s 28us/sample - loss: 0.5506 - ac

Epoch 14/100
44800/44800 [==============================] - 2s 38us/sample - loss: 0.5410 - accuracy: 0.7324 - val_loss: 0.5434 - val_accuracy: 0.7262
Epoch 15/100
44800/44800 [==============================] - 2s 42us/sample - loss: 0.5366 - accuracy: 0.7360 - val_loss: 0.5403 - val_accuracy: 0.7329
Epoch 16/100
44800/44800 [==============================] - 2s 43us/sample - loss: 0.5402 - accuracy: 0.7334 - val_loss: 0.5388 - val_accuracy: 0.7345
Epoch 17/100
44800/44800 [==============================] - 2s 40us/sample - loss: 0.5430 - accuracy: 0.7312 - val_loss: 0.5522 - val_accuracy: 0.7188
Epoch 18/100
44800/44800 [==============================] - 1s 31us/sample - loss: 0.5372 - accuracy: 0.7356 - val_loss: 0.5794 - val_accuracy: 0.7000
Epoch 19/100
44800/44800 [==============================] - 1s 31us/sample - loss: 0.5366 - accuracy: 0.7356 - val_loss: 0.5398 - val_accuracy: 0.7310
Epoch 20/100
44800/44800 [==============================] - 1s 31us/sample - loss: 0.5419 - ac

44800/44800 [==============================] - 2s 35us/sample - loss: 0.5349 - accuracy: 0.7362 - val_loss: 0.5477 - val_accuracy: 0.7295
Epoch 69/100
44800/44800 [==============================] - 1s 32us/sample - loss: 0.5392 - accuracy: 0.7334 - val_loss: 0.5753 - val_accuracy: 0.7021
Epoch 70/100
44800/44800 [==============================] - 2s 39us/sample - loss: 0.5355 - accuracy: 0.7357 - val_loss: 0.5613 - val_accuracy: 0.7153
Epoch 71/100
44800/44800 [==============================] - 1s 29us/sample - loss: 0.5464 - accuracy: 0.7265 - val_loss: 0.5548 - val_accuracy: 0.7233
Epoch 72/100
44800/44800 [==============================] - 1s 30us/sample - loss: 0.5402 - accuracy: 0.7329 - val_loss: 0.5566 - val_accuracy: 0.7251ss: 0.5395 - accuracy: 0.
Epoch 73/100
44800/44800 [==============================] - 1s 28us/sample - loss: 0.5357 - accuracy: 0.7358 - val_loss: 0.5717 - val_accuracy: 0.7063
Epoch 74/100
44800/44800 [==============================] - 1s 32us/sample - loss:

44800/44800 [==============================] - 2s 39us/sample - loss: 0.5317 - accuracy: 0.7363 - val_loss: 0.5456 - val_accuracy: 0.7312
Epoch 72/100
44800/44800 [==============================] - 1s 32us/sample - loss: 0.5324 - accuracy: 0.7365 - val_loss: 0.5399 - val_accuracy: 0.7341
Epoch 73/100
44800/44800 [==============================] - 2s 41us/sample - loss: 0.5277 - accuracy: 0.7412 - val_loss: 0.5487 - val_accuracy: 0.7297
Epoch 74/100
44800/44800 [==============================] - 1s 30us/sample - loss: 0.5331 - accuracy: 0.7371 - val_loss: 0.5472 - val_accuracy: 0.7276
Epoch 75/100
44800/44800 [==============================] - 2s 48us/sample - loss: 0.5394 - accuracy: 0.7304 - val_loss: 0.5754 - val_accuracy: 0.6996
Epoch 76/100
44800/44800 [==============================] - 2s 45us/sample - loss: 0.5328 - accuracy: 0.7360 - val_loss: 0.5484 - val_accuracy: 0.7275
Epoch 77/100
44800/44800 [==============================] - 2s 51us/sample - loss: 0.5286 - accuracy: 0.740

In [8]:
#============Test Phase============
yPred = model.predict(xTest)
yTest = to_categorical(yTest)
pred = np.argmax(yPred, axis=1)
test = np.argmax(yTest, axis=1)
showResults(test, pred)

              precision    recall  f1-score   support

    positive       0.72      0.79      0.75      6979
    negative       0.77      0.69      0.73      7021

    accuracy                           0.74     14000
   macro avg       0.74      0.74      0.74     14000
weighted avg       0.74      0.74      0.74     14000

Accuracy  : 0.7383571428571428
Precision : 0.7409640648250261
f1Score : 0.7377095732106843
[[5506 1473]
 [2190 4831]]


In [9]:
#========== Save the trained mode for future use ==========
model.save("trainedModel.h5")